In [1]:
'''
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from scipy.stats import linregress

# Import API key
from api_keys import weather_api_key

# Incorporated citipy to determine city based on latitude and longitude
from citipy import citipy

# Output File (CSV)
output_data_file = "output_data/cities.csv"

# Range of latitudes and longitudes
lat_range = (-90, 90)
lng_range = (-180, 180)


# List for holding lat_lngs and cities
lat_lngs = []
cities = []

# Create a set of random lat and lng combinations
lats = np.random.uniform(lat_range[0], lat_range[1], size=1500)
lngs = np.random.uniform(lng_range[0], lng_range[1], size=1500)
lat_lngs = zip(lats, lngs)

# Identify nearest city for each lat, lng combination
for lat_lng in lat_lngs:
    city = citipy.nearest_city(lat_lng[0], lat_lng[1]).city_name
    
    # If the city is unique, then add it to a our cities list
    if city not in cities:
        cities.append(city)

# Print the city count to confirm sufficient count
len(cities)


'''

'\n# Dependencies and Setup\nimport matplotlib.pyplot as plt\nimport pandas as pd\nimport numpy as np\nimport requests\nimport time\nfrom scipy.stats import linregress\n\n# Import API key\nfrom api_keys import weather_api_key\n\n# Incorporated citipy to determine city based on latitude and longitude\nfrom citipy import citipy\n\n# Output File (CSV)\noutput_data_file = "output_data/cities.csv"\n\n# Range of latitudes and longitudes\nlat_range = (-90, 90)\nlng_range = (-180, 180)\n\n\n# List for holding lat_lngs and cities\nlat_lngs = []\ncities = []\n\n# Create a set of random lat and lng combinations\nlats = np.random.uniform(lat_range[0], lat_range[1], size=1500)\nlngs = np.random.uniform(lng_range[0], lng_range[1], size=1500)\nlat_lngs = zip(lats, lngs)\n\n# Identify nearest city for each lat, lng combination\nfor lat_lng in lat_lngs:\n    city = citipy.nearest_city(lat_lng[0], lat_lng[1]).city_name\n    \n    # If the city is unique, then add it to a our cities list\n    if city not

In [2]:
# Import utilities
import pandas as pd
import requests
import json
from pprint import pprint

In [3]:
# Import charge station cities list
chargeStationCitiesdf = pd.read_csv("alt_fuel_stations_city_count.csv", encoding = "UTF-8", index_col = 0)
chargeStationCitiesdf.head(3)

,No. of Charge Stations
City,
ABBOTSFORD,1
ABERDEEN,11
ABILENE,3


In [4]:
# Import cities list with population density and coordinates and clean
uscitiesdf = pd.read_csv("uscities.csv", encoding = "UTF-8", index_col = 0)
uscitiesdf.head(3)

,city_ascii,state_id,state_name,county_fips,county_name,lat,lng,population,density,source,military,incorporated,timezone,ranking,zips,id
city,,,,,,,,,,,,,,,,
New York,New York,NY,New York,36061,New York,40.6943,-73.9249,18713220,10715.0,polygon,False,True,America/New_York,1,11229 11226 11225 11224 11222 11221 11220 1138...,1840034016
Los Angeles,Los Angeles,CA,California,6037,Los Angeles,34.1139,-118.4068,12750807,3276.0,polygon,False,True,America/Los_Angeles,1,90291 90293 90292 91316 91311 90037 90031 9000...,1840020491
Chicago,Chicago,IL,Illinois,17031,Cook,41.8373,-87.6862,8604203,4574.0,polygon,False,True,America/Chicago,1,60018 60649 60641 60640 60643 60642 60645 6064...,1840000494


In [6]:
#pull columns city_ascii, state_id, lat, lng
uscitiesColRemoved = uscitiesdf[["city_ascii","state_id","state_name","lat","lng","population","density"]]
# capitalize city_ascii
uscitiesColRemoved["city_ascii"] = uscitiesColRemoved["city_ascii"].str.upper()


C:\Users\sanab\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [8]:
# rename density column for clarification
uscitiesClean = uscitiesColRemoved.rename(columns = {"density":"Density(km^2)"}, copy = True)
uscitiesClean

,city_ascii,state_id,state_name,lat,lng,population,Density(km^2)
city,,,,,,,
New York,NEW YORK,NY,New York,40.6943,-73.9249,18713220,10715.0
Los Angeles,LOS ANGELES,CA,California,34.1139,-118.4068,12750807,3276.0
Chicago,CHICAGO,IL,Illinois,41.8373,-87.6862,8604203,4574.0
Miami,MIAMI,FL,Florida,25.7839,-80.2102,6445545,5019.0
Dallas,DALLAS,TX,Texas,32.7936,-96.7662,5743938,1526.0
...,...,...,...,...,...,...,...
Poso Park,POSO PARK,CA,California,35.8114,-118.6364,2,17.0
Lotsee,LOTSEE,OK,Oklahoma,36.1334,-96.2091,2,39.0
The Ranch,THE RANCH,MN,Minnesota,47.3198,-95.6952,2,2.0


In [129]:
# import gas prices per CA city and clean
CAgaspricesdf = pd.read_csv("CA_gas_prices_by_city.csv", encoding = "UTF-8", index_col = 0)
# re-index to pull city names into their own series
CAgaspricesIndexed = CAgaspricesdf.reset_index(drop = False)
# capitalize City
CAgaspricesIndexed["City"] = CAgaspricesIndexed["City"].str.upper()

CAgaspricesIndexed

,City,Price
0,BAKERSFIELD,$3.19
1,CHICO-PARADISE,$3.19
2,EL CENTRO,$3.19
3,FRESNO,$3.20
4,HANFORD,$3.13
5,CORCORAN,$3.13
6,LOS ANGELES,$3.28
7,LONG BEACH,$3.28
8,MADERA,$3.18
9,CHOWCHILLA,$3.18


In [143]:
# IS THIS NECESSARY??? Change to just simple counts for yelp responses?

# import city landmass list and clean
city_landmass = pd.read_csv("city_landmass.csv", encoding = "UTF-8", index_col = 0)
#pull columns city_ascii, state_id, lat, lng
city_landmassColRemoved = city_landmass[["city_cleaned","Land Area (sq km)"]]
# capitalize city_ascii
city_landmassColRemoved["city_cleaned"] = city_landmassColRemoved["city_cleaned"].str.upper()
city_landmassColRemoved

<ipython-input-143-43fc90834a6b>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  city_landmassColRemoved["city_cleaned"] = city_landmassColRemoved["city_cleaned"].str.upper()


,city_cleaned,Land Area (sq km)
Rank,,
1,NEW YORK,"8,936.00"
2,LOS ANGELES,"4,496.30"
3,CHICAGO,"6,326.70"
4,MIAMI,"3,208.00"
5,PHILADELPHIA,"5,131.70"
...,...,...
493,CHAMBERSBURG,96.8
494,GRANTS PASS,70.4
495,NEW BERN,112.4


In [ ]:
# merge station counts to the cities df as an inner merge
uscitiesStations = pd.merge(info_df, items_df, on="customer_id", how="left")

uscitiesClean

chargeStationCitiesdf
#

In [ ]:
api_key = "XzSNnJtfDK5IpktOiRC0ZE0n5I5jRh2UX6TEcURPglld794gH8y4vifAS6KaIp1LZnQFEq9sln52Soamph5Lk2v_IcztwvhLoohpjd1rXgqtIhk69shA63HRLMP8X3Yx"
headers = {"Authorization": "Bearer %s" % api_key}

In [101]:
url = "https://api.yelp.com/v3/businesses/search"
offset = 0
params = {"term": "McDonald's", "location":"San Diego", "limit":50, "offset": offset}

# ADD IN for each city in US
# For each response, if the name is "mcdonald's", add to mcd counter
#when you run into the first result where the name isn't mcdonald's
    #put counter into dataframe
    #move to next city
requestObjectpg1 = requests.get(url, params = params, headers = headers)
currentResult = 0
responseCount = len(response["businesses"])
currentResult = responseCount + offset #should be on result 50, next is 51
requestObjectpg1[]


print(f'Current result is: {currentResult}')
print(f'Response count is: {responseCount}')
print(f'Offset: {offset}')
print('The status code is {}'.format(requestObjectpg1.status_code))

#if results are 50, increment offset to get next object



Current result is: 50
Response count is: 50
Offset: 0
The status code is 200


In [102]:
requestObjectpg1.url

'https://api.yelp.com/v3/businesses/search?term=McDonald%27s&location=San+Diego&limit=50&offset=0'

In [103]:
if responseCount == 50:
    offset += currentResult + 1
    print (offset)
    params = {"term": "McDonald's", "location":"San Diego", "limit":50, "offset": offset}
    requestObjectpg2 = requests.get(url, params = params, headers = headers)
    responseCount = len(response["businesses"])
    currentResult = responseCount + offset
    currentResult
    print(f'Current result is: {currentResult}')
    print(f'Response count is: {responseCount}')
    print(f'Offset: {offset}')
    print('The status code is {}'.format(requestObjectpg2.status_code))

51
Current result is: 101
Response count is: 50
Offset: 51
The status code is 200


In [104]:
requestObjectpg2.url

'https://api.yelp.com/v3/businesses/search?term=McDonald%27s&location=San+Diego&limit=50&offset=51'

In [ ]:

    if responseCount == 50:
        offset += currentResult
        requestObjectpg3 = requests.get(url, params = params, headers = headers)


print('status code is {}'.format(requestObjectpg1.status_code))
print('status code is {}'.format(requestObjectpg2.status_code))
print('status code is {}'.format(requestObjectpg3.status_code))

#SHOW A PRETTY PRINT OF THIS

In [49]:
responseCount
offset

153

In [105]:
response1 = json.loads(requestObjectpg1.text)
pprint(response1)

{'businesses': [{'alias': 'mcdonalds-san-diego-17',
                 'categories': [{'alias': 'burgers', 'title': 'Burgers'},
                                {'alias': 'hotdogs', 'title': 'Fast Food'},
                                {'alias': 'coffee', 'title': 'Coffee & Tea'}],
                 'coordinates': {'latitude': 32.7857831249364,
                                 'longitude': -117.170181199908},
                 'display_phone': '(858) 560-6437',
                 'distance': 1596.1617584422195,
                 'id': 'KIZrAwB4KKhiQc-xx1pgHA',
                 'image_url': 'https://s3-media2.fl.yelpcdn.com/bphoto/Jn8Dg1RzPyLF2GdP9mmamg/o.jpg',
                 'is_closed': False,
                 'location': {'address1': '6945 Linda Vista Rd',
                              'address2': '',
                              'address3': '',
                              'city': 'San Diego',
                              'country': 'US',
                              'display_address

In [106]:
response2 = json.loads(requestObjectpg2.text)
pprint(response2)

{'businesses': [{'alias': 'mcdonalds-poway-2',
                 'categories': [{'alias': 'burgers', 'title': 'Burgers'},
                                {'alias': 'hotdogs', 'title': 'Fast Food'},
                                {'alias': 'coffee', 'title': 'Coffee & Tea'}],
                 'coordinates': {'latitude': 32.9524091,
                                 'longitude': -117.0598816},
                 'display_phone': '(858) 748-2636',
                 'distance': 20065.319008284212,
                 'id': 'A8hiOALp9bB-WRIawfOFSQ',
                 'image_url': 'https://s3-media2.fl.yelpcdn.com/bphoto/7VwCkjcsHUaAg0V_habgBA/o.jpg',
                 'is_closed': False,
                 'location': {'address1': '12510 Poway Rd',
                              'address2': '',
                              'address3': '',
                              'city': 'Poway',
                              'country': 'US',
                              'display_address': ['12510 Poway Rd',
   

In [45]:
response3 = json.loads(requestObjectpg2.text)
pprint(response3)

{'businesses': [{'alias': 'mcdonalds-san-diego-17',
                 'categories': [{'alias': 'burgers', 'title': 'Burgers'},
                                {'alias': 'hotdogs', 'title': 'Fast Food'},
                                {'alias': 'coffee', 'title': 'Coffee & Tea'}],
                 'coordinates': {'latitude': 32.7857831249364,
                                 'longitude': -117.170181199908},
                 'display_phone': '(858) 560-6437',
                 'distance': 1596.1617584422195,
                 'id': 'KIZrAwB4KKhiQc-xx1pgHA',
                 'image_url': 'https://s3-media2.fl.yelpcdn.com/bphoto/Jn8Dg1RzPyLF2GdP9mmamg/o.jpg',
                 'is_closed': False,
                 'location': {'address1': '6945 Linda Vista Rd',
                              'address2': '',
                              'address3': '',
                              'city': 'San Diego',
                              'country': 'US',
                              'display_address

In [46]:
# Compensate for search results going over 50
#if len(response["businesses"]) =50
print(len(response1["businesses"]))
print(len(response2["businesses"]))
print(len(response3["businesses"]))

50
50
50
